In [3]:
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

#preprocessing
def tokenize(sentence):
    return nltk.word_tokenize(sentence)


def stem(word):
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [4]:
# The neural network definition

import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)     #2 hidden layers
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)    #No ofclasses is the o/p 
        self.relu = nn.ReLU()                           #Relu is the activation func
    
    def forward(self, x):    #feeding forward
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [5]:
#nltk.download('punkt')

In [6]:
#training phase

import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#from nltk_utils import bag_of_words, tokenize, stem
#from model import NeuralNet

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag)) #mapping b/w i/p & o/p in training

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1500
batch_size = 11
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 10
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)  #length of the sample
        self.x_data = X_train          #Feature patterns stored in x_data
        self.y_data = y_train          # The output tagsstored in y_data

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  #if GPU support is available use 
                                                                        #it, either use the cpu
    
model = NeuralNet(input_size, hidden_size, output_size).to(device)   #Neuralnet is defined earlier
                                                                    #push our model to the device

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()   #to calculate the back propagation
        optimizer.step()
        
        #print epoch
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

#print final loss
print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)  #.pth means pytorch. It will save it to a pickle file 

print(f'training complete. file saved to {FILE}')

94 patterns
16 tags: ['Cold_drinks', 'French fries', 'chicken_momo', 'corn_momo', 'delivery', 'goodbye', 'greeting', 'items', 'menu_list', 'paneer_momo', 'payments', 'price', 'thanks', 'timing', 'veg_momo', 'wb']
100 unique stemmed words: ["'s", 'a', 'accept', 'allow', 'anyon', 'are', 'avail', 'buy', 'bye', 'can', 'card', 'cash', 'chicken', 'close', 'cod', 'cold', 'corn', 'cost', 'credit', 'day', 'deliveri', 'do', 'doe', 'drink', 'eat', 'food', 'french', 'fri', 'get', 'give', 'good', 'goodby', 'great', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'in', 'is', 'it', 'item', 'kind', 'later', 'list', 'long', 'look', 'lot', 'mastercard', 'me', 'menu', 'momo', 'morn', 'much', 'my', 'na', 'need', 'now', 'of', 'ok', 'okay', 'on', 'onli', 'open', 'order', 'paneer', 'pay', 'payment', 'paypal', 'price', 'procedur', 'remain', 'restur', 'see', 'sell', 'serv', 'ship', 'show', 'take', 'thank', 'that', 'the', 'there', 'time', 'to', 'varieti', 'veg', 'wan', 'want', 'what', 'when', 'which', 'will',

In [7]:
#The actual chat

import random
import json

import torch

#from model import NeuralNet
#from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #if GPU support is available use 
                                                                        #it, either use the cpu

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)   #Neuralnet is defined earlier
                                                                    #push our model to the device
model.load_state_dict(model_state)
model.eval()   #setting it to evaluation mode

bot_name = "Oreo"

def get_response(msg):
    sentence = tokenize(msg)             #msg is tokenised
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)   #converting to torch tensor

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)  #implemented softmax to get the actual probabilities
    prob = probs[0][predicted.item()]
    if prob.item() > 0.5:          #if p > 0.75, then only actual o/p comes
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    
    return "I do not understand..."


In [8]:
# UI Design using the Tkinter module

from tkinter import *

# Defining some colors and fonts in some variables
BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

# Creating a class for the main application
class ChatApplication:
    
    # Function for creating the main window
    def __init__(self):
        self.window = Tk()          # Tk() is a top-level widget which represents mostly the main window of an application
        self._setup_main_window()
        
    # Function for running the application
    def run(self):
        self.window.mainloop()
        
    # Function for setting up the widgets of the main window
    def _setup_main_window(self):
        
        # Setting up different attributes
        self.window.title("Oreo The Chatbot")
        self.window.resizable(width=True, height=True)
        self.window.configure(width=470, height=550, bg=BG_COLOR)        
        
        # sets default bg for top level windows
        self.tl_bg = "#EEEEEE"
        self.tl_bg2 = "#EEEEEE"
        self.tl_fg = "#000000"
        self.font = "Verdana 10"
        
        # menu and sub-menu bar
        main_menu = Menu(self.window)
        file_menu1 = Menu(self.window)
        file_menu2 = Menu(self.window)
        file_menu3 = Menu(self.window)
        font = Menu(self.window)
        color_theme = Menu(self.window)
        
        # Setting the submenu for the File menu
        file_menu1.add_command(label='Clear chat', command=self.clear_chat)
        file_menu1.add_command(label='Exit', command=self.chatexit)
        
        # Setting the submenu for the Options menu
        file_menu2.add_cascade(label='Font', menu=font)
        font.add_command(label="Default",command=self.font_change_default)
        font.add_command(label="Times",command=self.font_change_times)
        font.add_command(label="System",command=self.font_change_system)
        font.add_command(label="Helvetica",command=self.font_change_helvetica)
        font.add_command(label="Fixedsys",command=self.font_change_fixedsys)
        
        file_menu2.add_cascade(label='Color theme', menu=color_theme)
        color_theme.add_command(label="Default",command=self.color_theme_default)
        color_theme.add_command(label="Grey",command=self.color_theme_grey) 
        color_theme.add_command(label="Blue",command=self.color_theme_dark_blue)        
        color_theme.add_command(label="Torque",command=self.color_theme_turquoise)
        color_theme.add_command(label="Hacker",command=self.color_theme_hacker)
        
        # Setting the submenu for the Help menu
        file_menu3.add_command(label='Developers', command=self.about)
        file_menu3.add_command(label='About Oreo', command=self.msg)
        
        # Setting the three main menus in the menu bar
        main_menu.add_cascade(label='File', menu=file_menu1)
        main_menu.add_cascade(label='Options', menu=file_menu2)
        main_menu.add_cascade(label='Help', menu=file_menu3)
        self.window.configure(menu=main_menu)  
        
        
        # setting the head label
        head_label = Label(self.window, bg="#dbb9eb", fg="#000000",
                           text="WELCOME", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)
        
        # setting the tiny divider below the head label
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # setting the text widget
        self.text_widget = Text(self.window, width=20, height=2, bg="#500373", fg=TEXT_COLOR,
                                font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        # setting the scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        # setting the bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        # setting the message entry box
        self.msg_entry = Entry(bottom_label, bg="#210130", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)
        
        # setting the send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg="#f0d7fc",
                             command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)
        
    
    # defining a function for the Clear Chat option in the File menu
    def clear_chat(self):
        self.text_widget.config(state=NORMAL)
        self.text_widget.delete(1.0, END)
        self.text_widget.delete(1.0, END)
        self.text_widget.config(state=DISABLED)
    
    # defining a function for the Exit option in the File menu
    def chatexit(self):
        exit()
    
    # defining a function for the About Oreo option in the Help menu
    def msg(self):
        messagebox.showinfo("Oreo The Chatbot v1.0",'Oreo is a chatbot for answering queries related to online food delivery for a restaurant.\nIt is based on retrival-based NLP using pythons NLTK tool-kit module\nGUI is based on Tkinter')
    
    # defining a function for the Developers option in the Help menu
    def about(self):
        messagebox.showinfo("Oreo Developers","1. Arpan Maity\n2. Srijita Guha Roy")
        
        
    # defining five functions for the changing the font option in the Options menu   
    def font_change_default(self):                            # To change the font to Verdana 10
        self.text_widget.config(font="Verdana 10")
        self.msg_entry.config(font="Verdana 10")
        self.font = "Verdana 10"

    def font_change_times(self):
        self.text_widget.config(font="Times")                 # To change the font to Times
        self.msg_entry.config(font="Times")
        self.font = "Times"

    def font_change_system(self):
        self.text_widget.config(font="System")                # To change the font to System
        self.msg_entry.config(font="System")
        self.font = "System"

    def font_change_helvetica(self):
        self.text_widget.config(font="helvetica 10")          # To change the font to Helvetica 10
        self.msg_entry.config(font="helvetica 10")
        self.font = "helvetica 10"

    def font_change_fixedsys(self):
        self.text_widget.config(font="fixedsys")              # To change the font to Fixedsys
        self.msg_entry.config(font="fixedsys")
        self.font = "fixedsys"
        
    # defining five functions for the changing the color theme in the Options menu  
    # Default color theme
    def color_theme_default(self):
        self.window.config(bg="#2a2b2d")
        self.text_widget.config(bg="#500373")
        self.text_widget.config(bg="#500373", fg="#FFFFFF")
        self.msg_entry.config(bg="#210130", fg="#FFFFFF", insertbackground="#FFFFFF")

        self.tl_bg = "#212121"
        self.tl_bg2 = "#2a2b2d"
        self.tl_fg = "#FFFFFF"

    # Grey color theme
    def color_theme_grey(self):
        self.window.config(bg="#444444")
        self.text_widget.config(bg="#444444")
        self.text_widget.config(bg="#4f4f4f", fg="#ffffff")
        self.msg_entry.config(bg="#4f4f4f", fg="#ffffff", insertbackground="#ffffff")

        self.tl_bg = "#4f4f4f"
        self.tl_bg2 = "#444444"
        self.tl_fg = "#ffffff"


    # Blue color theme
    def color_theme_dark_blue(self):
        self.window.config(bg="#263b54")
        self.text_widget.config(bg="#263b54")
        self.text_widget.config(bg="#1c2e44", fg="#FFFFFF")
        self.msg_entry.config(bg="#1c2e44", fg="#FFFFFF", insertbackground="#FFFFFF")

        self.tl_bg = "#1c2e44"
        self.tl_bg2 = "#263b54"
        self.tl_fg = "#FFFFFF" 
    

    # Torque color theme
    def color_theme_turquoise(self):
        self.window.config(bg="#003333")
        self.text_widget.config(bg="#003333")
        self.text_widget.config(bg="#669999", fg="#FFFFFF")
        self.msg_entry.config(bg="#669999", fg="#FFFFFF", insertbackground="#FFFFFF")

        self.tl_bg = "#669999"
        self.tl_bg2 = "#003333"
        self.tl_fg = "#FFFFFF"

    # Hacker color theme
    def color_theme_hacker(self):
        self.window.config(bg="#0F0F0F")
        self.text_widget.config(bg="#0F0F0F")
        self.text_widget.config(bg="#0F0F0F")
        self.msg_entry.config(bg="#0F0F0F", fg="#33FF33", insertbackground="#33FF33")

        self.tl_bg = "#0F0F0F"
        self.tl_bg2 = "#0F0F0F"
        self.tl_fg = "#33FF33"
    
    # Function to input the message from the user
    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()              # To get the input meassage from the user as a string
        self._insert_message(msg, "You")
    
    # Function to insert the message from the message entry box to the chat area
    def _insert_message(self, msg, sender):
        if not msg:
            return
        
        # To delete the message from the message entry box after it has been sent
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"                    # format to print the user entered message in the text widget area
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        msg2 = f"{bot_name}: {get_response(msg)}\n\n"    # format to print the bot message in the text widget area
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)
        
        self.text_widget.see(END)                       # will automatically scroll to the end so that we are always able to see the last message
             
# main function to run the chat application       
if __name__ == "__main__":
    app = ChatApplication()
    app.run()